In [1]:
import pandas as pd
import os
import requests
from datetime import timedelta, date
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\LEO PALO-IT\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
# EL EJEMPLO ORIGINAL TENIA EL AÑO 2017, MES 1, DIA 1, LO QUE SE HIZO
# AQUÍ FUE LIMITAR LA FECHA PARA HACER UNA PRUEBA.
# OTRA COSA A CONSIDERAR SON LOS PAÍSES, EL PROGRAMA HACE UN SCRAPING
# Y TRAE TODOS, LO QUE SE HIZO FUE LIMITAR SOLO A ESTADOS UNIDOS PARA
# QUE LA CONSULTA NO TARDARA TANTO.
# PUEDEN VER EL RESULTADO DEL SCRAPING EN LA CARPETA DATA
# EJ. data/united_states.csv
year = 2021
month = 7
day = 10

# It generates a list of dates between Jan 1, 2017 and today
# in YYYY-MM-DD format
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# It creates the list of page links we will get the data from.
def create_links(country):
    start_date = date(year, month, day)
    end_date = pd.datetime.today().date()
    links = []
    dates = daterange(start_date, end_date)
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links, dates)


# It reads the webpage.
def get_webpage(link):
    browser.visit(link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    return(soup)

# It collects the data for each country, and write them in a list.
# The entries are (in order): Song, Artist, Date, Play Count, Rank
def get_data(country):
    [links, dates] = create_links(country);
    rows = []

    for (link, date) in zip(links, dates):
        soup = get_webpage(link)
        entries = soup.find_all("td", class_ = "chart-table-track")
        streams = soup.find_all("td", class_="chart-table-streams")
        for i, (entry, stream) in enumerate(zip(entries,streams)):
            song = entry.find('strong').get_text()
            artist = entry.find('span').get_text()[3:]
            play_count = stream.get_text()
            rows.append([song, artist, date, play_count, i+1])

    return(rows)

# It exports the data for each country in a csv format.
# The column names are Song, Artist, Date, Streams, Rank.
def save_data(country):
    if not os.path.exists('data'):
        os.makedirs('data')
    file_name = 'data/' + country[1].replace(" ", "_").lower() + '.csv'
    data = get_data(country[0])
    if(len(data)!= 0):
        data = pd.DataFrame(data, columns=['Song','Artist','Date', 'Streams','Rank'])
        data.to_csv(file_name, sep=',', float_format='%s', index = False)

# It generates a list of countries for which the data is provided.
def get_countries():
    url = 'https://spotifycharts.com/regional'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    countries = []
    ctys = soup.find('ul').findAll("li")
    for cty in ctys:
        countries.append([cty["data-value"],cty.get_text()])
    return(countries)

# It runs the function save_data for each country.
# In other words, it creates the .csv data files for each country.
def scrape_data():
    #countries = get_countries()
    countries = [['us', 'United States']]
    print(countries)
    for country in countries:
        save_data(country)

    
    
scrape_data()

[['us', 'United States']]


D:\Anaconda3\envs\Python_01\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  
